In [1]:
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.concat([
    pd.read_csv('../data/feature_selection_negative.csv', index_col=0),
    pd.read_csv('../data/decomp_neg.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=1, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
248/248 [==============================] - 1s 5ms/step - loss: 1.8921 - acc: 0.2460
Epoch 2/100
248/248 [==============================] - 0s 62us/step - loss: 2.1679 - acc: 0.2702
Epoch 3/100
248/248 [==============================] - 0s 57us/step - loss: 1.9129 - acc: 0.3105
Epoch 4/100
248/248 [==============================] - 0s 50us/step - loss: 1.7990 - acc: 0.2863
Epoch 5/100
248/248 [==============================] - 0s 49us/step - loss: 1.9004 - acc: 0.2903
Epoch 6/100
248/248 [==============================] - 0s 48us/step - loss: 1.8269 - acc: 0.3145
Epoch 7/100
248/248 [==============================] - 0s 45us/step - loss: 1.6689 - acc: 0.3508
Epoch 8/100
248/248 [==============================] - 0s 45us/step - loss: 1.7509 - acc: 0.3347
Epoch 9/100
248/248 [==============================] - 0s 50us/step - loss: 1.7182 - acc: 0.3266
Epoch 10/100
248/248 [==============================] - 0s 55us/step - loss: 1.6068 - acc: 0.3508
Epoch 11/100
248/248 [========

Epoch 85/100
248/248 [==============================] - 0s 52us/step - loss: 0.9911 - acc: 0.5927
Epoch 86/100
248/248 [==============================] - 0s 45us/step - loss: 1.0404 - acc: 0.5524
Epoch 87/100
248/248 [==============================] - 0s 45us/step - loss: 1.0268 - acc: 0.5685
Epoch 88/100
248/248 [==============================] - 0s 45us/step - loss: 1.0896 - acc: 0.5444
Epoch 89/100
248/248 [==============================] - 0s 51us/step - loss: 0.9204 - acc: 0.6048
Epoch 90/100
248/248 [==============================] - 0s 52us/step - loss: 1.0677 - acc: 0.5484
Epoch 91/100
248/248 [==============================] - 0s 51us/step - loss: 1.0192 - acc: 0.5685
Epoch 92/100
248/248 [==============================] - 0s 47us/step - loss: 0.9768 - acc: 0.5565
Epoch 93/100
248/248 [==============================] - 0s 55us/step - loss: 0.9718 - acc: 0.6089
Epoch 94/100
248/248 [==============================] - 0s 50us/step - loss: 0.9972 - acc: 0.5565
Epoch 95/100
248/248

Epoch 68/100
249/249 [==============================] - 0s 54us/step - loss: 1.1368 - acc: 0.5181
Epoch 69/100
249/249 [==============================] - 0s 47us/step - loss: 1.1511 - acc: 0.4900
Epoch 70/100
249/249 [==============================] - 0s 51us/step - loss: 1.1645 - acc: 0.5100
Epoch 71/100
249/249 [==============================] - 0s 51us/step - loss: 1.1665 - acc: 0.4578
Epoch 72/100
249/249 [==============================] - 0s 65us/step - loss: 1.1649 - acc: 0.5020
Epoch 73/100
249/249 [==============================] - 0s 59us/step - loss: 1.1356 - acc: 0.5100
Epoch 74/100
249/249 [==============================] - 0s 55us/step - loss: 1.1656 - acc: 0.4819
Epoch 75/100
249/249 [==============================] - 0s 55us/step - loss: 1.1217 - acc: 0.5301
Epoch 76/100
249/249 [==============================] - 0s 61us/step - loss: 1.0659 - acc: 0.5502
Epoch 77/100
249/249 [==============================] - 0s 54us/step - loss: 1.1608 - acc: 0.5462
Epoch 78/100
249/249

Epoch 51/100
249/249 [==============================] - 0s 44us/step - loss: 1.2175 - acc: 0.4538
Epoch 52/100
249/249 [==============================] - 0s 53us/step - loss: 1.1538 - acc: 0.5100
Epoch 53/100
249/249 [==============================] - 0s 47us/step - loss: 1.2222 - acc: 0.4819
Epoch 54/100
249/249 [==============================] - 0s 47us/step - loss: 1.0431 - acc: 0.5301
Epoch 55/100
249/249 [==============================] - 0s 50us/step - loss: 1.2197 - acc: 0.4739
Epoch 56/100
249/249 [==============================] - 0s 58us/step - loss: 1.1868 - acc: 0.4980
Epoch 57/100
249/249 [==============================] - 0s 55us/step - loss: 1.1655 - acc: 0.4859
Epoch 58/100
249/249 [==============================] - 0s 59us/step - loss: 1.2416 - acc: 0.4378
Epoch 59/100
249/249 [==============================] - 0s 45us/step - loss: 1.2571 - acc: 0.4257
Epoch 60/100
249/249 [==============================] - 0s 48us/step - loss: 1.0633 - acc: 0.5141
Epoch 61/100
249/249

Epoch 34/100
248/248 [==============================] - 0s 63us/step - loss: 0.4644 - acc: 0.8105
Epoch 35/100
248/248 [==============================] - 0s 65us/step - loss: 0.4140 - acc: 0.8347
Epoch 36/100
248/248 [==============================] - 0s 59us/step - loss: 0.5189 - acc: 0.7823
Epoch 37/100
248/248 [==============================] - 0s 69us/step - loss: 0.5901 - acc: 0.7903
Epoch 38/100
248/248 [==============================] - 0s 70us/step - loss: 0.5114 - acc: 0.7903
Epoch 39/100
248/248 [==============================] - 0s 66us/step - loss: 0.5439 - acc: 0.8105
Epoch 40/100
248/248 [==============================] - 0s 61us/step - loss: 0.5527 - acc: 0.7702
Epoch 41/100
248/248 [==============================] - 0s 61us/step - loss: 0.5809 - acc: 0.7823
Epoch 42/100
248/248 [==============================] - 0s 64us/step - loss: 0.5224 - acc: 0.8185
Epoch 43/100
248/248 [==============================] - 0s 62us/step - loss: 0.4924 - acc: 0.8024
Epoch 44/100
248/248

...

Epoch 31/100
248/248 [==============================] - 0s 71us/step - loss: 0.4738 - acc: 0.8387
Epoch 32/100
248/248 [==============================] - 0s 73us/step - loss: 0.4570 - acc: 0.8105
Epoch 33/100
248/248 [==============================] - 0s 69us/step - loss: 0.4103 - acc: 0.8306
Epoch 34/100
248/248 [==============================] - 0s 68us/step - loss: 0.4041 - acc: 0.8427
Epoch 35/100
248/248 [==============================] - 0s 71us/step - loss: 0.4298 - acc: 0.8065
Epoch 36/100
248/248 [==============================] - 0s 78us/step - loss: 0.3904 - acc: 0.8508
Epoch 37/100
248/248 [==============================] - 0s 79us/step - loss: 0.3775 - acc: 0.8508
Epoch 38/100
248/248 [==============================] - 0s 77us/step - loss: 0.3896 - acc: 0.8508
Epoch 39/100
248/248 [==============================] - 0s 72us/step - loss: 0.3721 - acc: 0.8669
Epoch 40/100
248/248 [==============================] - 0s 70us/step - loss: 0.3625 - acc: 0.8589
Epoch 41/100
248/248

Epoch 14/100
249/249 [==============================] - 0s 85us/step - loss: 0.7142 - acc: 0.6948
Epoch 15/100
249/249 [==============================] - 0s 76us/step - loss: 0.6323 - acc: 0.7470
Epoch 16/100
249/249 [==============================] - 0s 77us/step - loss: 0.6577 - acc: 0.7590
Epoch 17/100
249/249 [==============================] - 0s 77us/step - loss: 0.6414 - acc: 0.7390
Epoch 18/100
249/249 [==============================] - 0s 80us/step - loss: 0.5902 - acc: 0.7510
Epoch 19/100
249/249 [==============================] - 0s 74us/step - loss: 0.5519 - acc: 0.7631
Epoch 20/100
249/249 [==============================] - 0s 81us/step - loss: 0.5492 - acc: 0.7671
Epoch 21/100
249/249 [==============================] - 0s 80us/step - loss: 0.5165 - acc: 0.7671
Epoch 22/100
249/249 [==============================] - 0s 80us/step - loss: 0.4774 - acc: 0.7952
Epoch 23/100
249/249 [==============================] - 0s 69us/step - loss: 0.5153 - acc: 0.7590
Epoch 24/100
249/249

Epoch 98/100
249/249 [==============================] - 0s 75us/step - loss: 0.2453 - acc: 0.8956
Epoch 99/100
249/249 [==============================] - 0s 79us/step - loss: 0.2379 - acc: 0.9036
Epoch 100/100
249/249 [==============================] - 0s 30us/step
Epoch 1/100
249/249 [==============================] - 4s 15ms/step - loss: 2.0271 - acc: 0.2450
Epoch 2/100
249/249 [==============================] - 0s 71us/step - loss: 2.3439 - acc: 0.4016
Epoch 3/100
249/249 [==============================] - 0s 72us/step - loss: 1.6494 - acc: 0.4016
Epoch 4/100
249/249 [==============================] - 0s 78us/step - loss: 1.8580 - acc: 0.4779
Epoch 5/100
249/249 [==============================] - 0s 78us/step - loss: 1.8582 - acc: 0.4378
Epoch 6/100
249/249 [==============================] - 0s 72us/step - loss: 1.6138 - acc: 0.4739
Epoch 7/100
249/249 [==============================] - 0s 76us/step - loss: 1.1350 - acc: 0.5382
Epoch 8/100
249/249 [==============================] - 

Epoch 81/100
249/249 [==============================] - 0s 69us/step - loss: 0.5039 - acc: 0.8233
Epoch 82/100
249/249 [==============================] - 0s 66us/step - loss: 0.2963 - acc: 0.8876
Epoch 83/100
249/249 [==============================] - 0s 68us/step - loss: 0.3058 - acc: 0.8916
Epoch 84/100
249/249 [==============================] - 0s 70us/step - loss: 0.2314 - acc: 0.8956
Epoch 85/100
249/249 [==============================] - 0s 70us/step - loss: 0.2163 - acc: 0.9157
Epoch 86/100
249/249 [==============================] - 0s 80us/step - loss: 0.2435 - acc: 0.8835
Epoch 87/100
249/249 [==============================] - 0s 81us/step - loss: 0.2598 - acc: 0.8876
Epoch 88/100
249/249 [==============================] - 0s 73us/step - loss: 0.2934 - acc: 0.8675
Epoch 89/100
249/249 [==============================] - 0s 72us/step - loss: 0.3189 - acc: 0.8876
Epoch 90/100
249/249 [==============================] - 0s 78us/step - loss: 0.2877 - acc: 0.9036
Epoch 91/100
249/249

Epoch 64/100
373/373 [==============================] - 0s 50us/step - loss: 0.3441 - acc: 0.8713
Epoch 65/100
373/373 [==============================] - 0s 48us/step - loss: 0.4405 - acc: 0.8177
Epoch 66/100
373/373 [==============================] - 0s 48us/step - loss: 0.6895 - acc: 0.8123
Epoch 67/100
373/373 [==============================] - 0s 45us/step - loss: 0.4847 - acc: 0.7909
Epoch 68/100
373/373 [==============================] - 0s 47us/step - loss: 0.5100 - acc: 0.8016
Epoch 69/100
373/373 [==============================] - 0s 46us/step - loss: 0.4046 - acc: 0.8231
Epoch 70/100
373/373 [==============================] - 0s 47us/step - loss: 0.3789 - acc: 0.8365
Epoch 71/100
373/373 [==============================] - 0s 46us/step - loss: 0.3289 - acc: 0.8579
Epoch 72/100
373/373 [==============================] - 0s 51us/step - loss: 0.3335 - acc: 0.8606
Epoch 73/100
373/373 [==============================] - 0s 48us/step - loss: 0.3332 - acc: 0.8472
Epoch 74/100
373/373

In [9]:
t = grid_result.best_estimator_

In [10]:
t.score(X_test, y_test)

94/94 [==============================] - 1s 13ms/step


0.6595744490623474

In [11]:
t.model.save('../model/Keras_best_params_neg_dec.h5')